ETABS DESIGN PIERS AS COLUMNS

Introduction:
- Tool is to design walls as columns to AS3600:2018 Section 10 for ULS load combinations
- ETABS model should be correct and run without errors
- Pier labels should be assigned to walls manually
- Piers assigned should be straight walls only, no T or L shaped walls are permitted
- Two walls meeting at a corner or junction should be assigned two separate pier labels
- Program will design the minimum bar size for the bar spacing selected in each direction
- This may not necessarily comply with code minimum requirements. This is to be checked manually
- X-X is defined as strong direction and Y-Y is to be the weak direction

Load combinations:
- ETABS load combinations must be set up as follows:
- eq_env_1 -> Envelope of load combinations with unfactored earthquake load for flexural design
- eq_env_2 -> Envelope of load combinations with factored earthquake load for shear design
- wind_env -> Envelope of all ULS wind combinations
- If designing non-ductile walls for earthquake actions, eq_env_1 and eq_env_2 will be the same
    

In [ ]:
import sys
sys.path.append(r'C:\_Github\structural_engineering_toolbox')
from etabs_tools import etabs_api
from design_functions import as3600_column_design
from design_reports import design_reports
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Define buttons
connect_button = widgets.Button(description='Connect to ETABS', tooltip='Make sure ETABS model is open and working')
design_button = widgets.Button(description='Design piers', tooltip='Make sure ETABS model is open and working')

# Create output widgets
output_1 = widgets.Output()  # For dropdowns
output_2 = widgets.Output()  # For results

# Dictionary for dropdowns
dropdowns = {}

# Initialize ETABS API
etabs_api_instance = etabs_api.etabs_api()

# Callback for connect_button (fetch load cases and create dropdowns)
def on_connect_button_clicked(b):
    with output_1:
        clear_output()  # Clear previous widgets

        # Fetch load cases from ETABS
        try:
            load_case_list = etabs_api_instance.get_load_cases()
        except Exception as e:
            print(f"Error fetching load cases: {e}")
            return
        
        # Create dropdowns for load cases
        dropdowns["eq_env_1"] = widgets.Dropdown(options=load_case_list, description='Unfactored EQ Envelope')
        dropdowns["eq_env_2"] = widgets.Dropdown(options=load_case_list, description='Factored EQ Envelope')
        dropdowns["wind_env"] = widgets.Dropdown(options=load_case_list, description='Wind Envelope')

        # Reinforcement spacing options
        spacing_options = [150, 200, 250, 300]
        dropdowns["vertical_spacing"] = widgets.Dropdown(options=spacing_options, value=200, description='Vertical Spacing')
        dropdowns["horizontal_spacing"] = widgets.Dropdown(options=spacing_options, value=200, description='Horizontal Spacing')

        # Out-of-plane checkbox
        dropdowns['out_of_plane'] = widgets.Checkbox(value=False, description='Design out of plane bending')

        # Display dropdowns and the design button
        display(
            dropdowns["eq_env_1"],
            dropdowns["eq_env_2"],
            dropdowns["wind_env"],
            dropdowns["vertical_spacing"],
            dropdowns["horizontal_spacing"],
            dropdowns['out_of_plane'],
            design_button
        )
    with output_2:
        clear_output()

# Attach callback to connect button
connect_button.on_click(on_connect_button_clicked)

# Progress bar widget
status_label = widgets.Label(value="")

# Callback for design_button (run pier design and display results)
def on_design_button_clicked(b):
    with output_2:
        clear_output()  # Clear previous output
        status_label.value = "Processing... Please wait."
        display(status_label)
        
        # Ensure dropdowns are populated before proceeding
        if not dropdowns:
            print("Error: Please connect to ETABS first.")
            return

        # Fetch pier data from ETABS
        try:
            piers = etabs_api_instance.get_piers(load_cases=[
                dropdowns['eq_env_1'].value,
                dropdowns['eq_env_2'].value,
                dropdowns['wind_env'].value
            ])
        except Exception as e:
            print(f"Error fetching pier data: {e}")
            return

        # Run design function
        try:
            designed_piers_df = as3600_column_design.design_all_piers(
                piers=piers,
                eq_env_1=dropdowns['eq_env_1'].value,
                eq_env_2=dropdowns['eq_env_2'].value,
                wind_env=dropdowns["wind_env"].value,
                vertical_spacing=dropdowns['vertical_spacing'].value,
                horizontal_spacing=dropdowns['horizontal_spacing'].value
            )

            # Show completed progress
            status_label.value = "Design complete"

            # Display completed df
            display(designed_piers_df)

        except Exception as e:
            print(f"Error in design function: {e}")
            status_label.value = ""

# Attach callback to design button
design_button.on_click(on_design_button_clicked)

# Display connect button and output areas
display(connect_button, output_1, output_2)
